데이터 전처리

In [5]:
import pandas as pd

# Load dataset
file_path = 'kma_2022_andong_noNULL_noDuplicated.csv'  # 파일 경로를 지정하세요.
data = pd.read_csv(file_path)

# Convert 'TM' column to datetime format
data['TM'] = pd.to_datetime(data['TM'], format='%Y%m%d%H%M')

# Select relevant columns for prediction
selected_columns = ['STN','TM', 'WD', 'WS', 'PA', 'PS', 'TA', 'TD', 'HM', 'CA_TOT', 'CA_MID', 'VS', 'TS', 'TE_005', 'TE_01', 'TE_02','TE_03', 'PV', 'PM10']
processed_data = data[selected_columns]

# Sort by time
processed_data = processed_data.sort_values('TM').reset_index(drop=True)

BASE MODEL : ARIMA 모델

In [6]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

arima_data = processed_data[['TM', 'PM10']].set_index('TM')

train_size = int(len(arima_data) * 0.7)
val_size = int(len(arima_data) * 0.15)

train_arima = arima_data.iloc[:train_size]['PM10']
y_val = arima_data.iloc[train_size:train_size + val_size]['PM10']
y_test = arima_data.iloc[train_size + val_size:]['PM10']
arima_order = (5, 1, 0)  # (p, d, q) 값 조정 가능
arima_model = ARIMA(train_arima, order=arima_order)
arima_fit = arima_model.fit()

# 예측
forecast_val = arima_fit.forecast(steps=len(y_val))
forecast_test = arima_fit.forecast(steps=len(y_test))

# 평가
val_rmse = np.sqrt(mean_squared_error(y_val, forecast_val))
val_mae = mean_absolute_error(y_val, forecast_val)
test_rmse = np.sqrt(mean_squared_error(y_test, forecast_test))
test_mae = mean_absolute_error(y_test, forecast_test)

print(f"ARIMA Validation RMSE: {val_rmse}, MAE: {val_mae}")
print(f"ARIMA Test RMSE: {test_rmse}, MAE: {test_mae}")


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


ARIMA Validation RMSE: 13.702848237284295, MAE: 10.553956455224085
ARIMA Test RMSE: 24.007840304954517, MAE: 20.149693776969283


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


MLP model

In [7]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Features (X) and target (y)
X = processed_data.drop(columns=['TM', 'PM10'])
y = processed_data['PM10']

# Split into training, validation, and test sets
train_size = 0.7
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=train_size, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)


# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# MLP 모델 설계
mlp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer
])

mlp_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 학습
history = mlp_model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=50,
    batch_size=32,
    verbose=1
)

# 평가
train_loss = history.history['loss'][-1]  # 마지막 epoch의 train loss
val_loss, val_mae = mlp_model.evaluate(X_val_scaled, y_val, verbose=0)
test_loss, test_mae = mlp_model.evaluate(X_test_scaled, y_test, verbose=0)

print(f"MLP Train Loss (RMSE): {np.sqrt(train_loss)}")
print(f"MLP Validation Loss (RMSE): {np.sqrt(val_loss)}, MAE: {val_mae}")
print(f"MLP Test Loss (RMSE): {np.sqrt(test_loss)}, MAE: {test_mae}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 472.8409 - mae: 16.0177 - val_loss: 121.8925 - val_mae: 8.7467
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 155.7781 - mae: 8.6523 - val_loss: 113.3746 - val_mae: 8.4778
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 141.2676 - mae: 8.3283 - val_loss: 118.7684 - val_mae: 8.6949
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 121.9655 - mae: 7.8959 - val_loss: 132.1961 - val_mae: 9.1647
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 111.9840 - mae: 7.6048 - val_loss: 141.5859 - val_mae: 9.4207
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 110.5999 - mae: 7.4389 - val_loss: 156.0567 - val_mae: 9.8958
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 103.2934 - mae: 7.3368 - val_loss: 160.8002 - val_mae: 9.9788
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 98.6247 - mae: 7.1312 - val_loss: 170.0202 - val_mae: 10.2316
Epoch 9/50
187/187 ━━━━

In [8]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Features (X) and target (y)
X = processed_data.drop(columns=['TM', 'PM10'])
y = processed_data['PM10']

# Split into training, validation, and test sets
train_size = 0.7
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=train_size, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)


# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# MLP 모델 설계
mlp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer
])

mlp_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 학습
history = mlp_model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=200,
    batch_size=32,
    verbose=1
)

# 평가
train_loss = history.history['loss'][-1]  # 마지막 epoch의 train loss
val_loss, val_mae = mlp_model.evaluate(X_val_scaled, y_val, verbose=0)
test_loss, test_mae = mlp_model.evaluate(X_test_scaled, y_test, verbose=0)

print(f"MLP Train Loss (RMSE): {np.sqrt(train_loss)}")
print(f"MLP Validation Loss (RMSE): {np.sqrt(val_loss)}, MAE: {val_mae}")
print(f"MLP Test Loss (RMSE): {np.sqrt(test_loss)}, MAE: {test_mae}")

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 427.5121 - mae: 15.2061 - val_loss: 133.2693 - val_mae: 9.1257
Epoch 2/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 143.2215 - mae: 8.4794 - val_loss: 111.7285 - val_mae: 8.4143
Epoch 3/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 133.8654 - mae: 8.2556 - val_loss: 114.4147 - val_mae: 8.4275
Epoch 4/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 120.1380 - mae: 7.8382 - val_loss: 115.1718 - val_mae: 8.4566
Epoch 5/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 119.1953 - mae: 7.7500 - val_loss: 135.4169 - val_mae: 9.2224
Epoch 6/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 114.9957 - mae: 7.5424 - val_loss: 148.2520 - val_mae: 9.6362
Epoch 7/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 118.2020 - mae: 7.6586 - val_loss: 163.1243 - val_mae: 10.1011
Epoch 8/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 103.2066 - mae: 7.2351 - val_loss: 185.0685 - val_mae: 10.6768
Epoch 9/200
187/187 ━━━━━